In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [46]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=9,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 759,556 || all params: 125,408,264 || trainable%: 0.6057


In [47]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [48]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.368000,1.323395,0.698438
200,0.967000,0.400327,0.875000
300,0.371700,0.347522,0.881250
400,0.311000,0.336938,0.895312
500,0.315400,0.323079,0.901563
600,0.303600,0.328151,0.901563
700,0.312200,0.315927,0.901563
800,0.313000,0.305652,0.907813
900,0.291600,0.312534,0.906250
1000,0.287700,0.308664,0.904687


In [50]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-1200'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:58<00:00,  4.25it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-1200.csv


In [49]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3680,1.323395,0.366250,0.698438,0.000048,0.013405,0.044605,1.033705,-0.332188,0.524385
1,200,0.9670,0.400327,0.746875,0.875000,0.000045,0.026810,0.566673,2.415527,-0.128125,0.853571
2,300,0.3717,0.347522,0.875446,0.881250,0.000043,0.040214,0.024178,1.069572,-0.005804,0.993414
3,400,0.3110,0.336938,0.890625,0.895312,0.000040,0.053619,-0.025938,0.923018,-0.004687,0.994764
4,500,0.3154,0.323079,0.892857,0.901563,0.000037,0.067024,-0.007679,0.976232,-0.008705,0.990344
5,600,0.3036,0.328151,0.900000,0.901563,0.000035,0.080429,-0.024551,0.925185,-0.001563,0.998267
6,700,0.3122,0.315927,0.902232,0.901563,0.000033,0.093834,-0.003727,0.988204,0.000670,1.000743
7,800,0.3130,0.305652,0.892411,0.907813,0.000030,0.107239,0.007348,1.024042,-0.015402,0.983034
8,900,0.2916,0.312534,0.901786,0.906250,0.000028,0.120643,-0.020934,0.933019,-0.004464,0.995074
9,1000,0.2877,0.308664,0.904911,0.904687,0.000025,0.134048,-0.020964,0.932083,0.000223,1.000247
